# 1. Extracción de datos desde la API de RAWG

Este notebook contiene la función encargada de extraer los datos desde la API pública de RAWG, la cual será utilizada por todos los miembros del grupo para responder a sus respectivas subpreguntas.

La función `obtener_juegos_por_anio()` permite recopilar información de videojuegos publicados entre un rango de años, devolviendo un DataFrame con los campos proporcionados por la API.

Además, la función `juegos_json()` permite guardar los resultados en un archivo `.csv` dentro del directorio `../data/raw/` para su posterior procesamiento.


In [ ]:
import requests
import pandas as pd
from tqdm.notebook import tqdm
import time
import os

In [ ]:
API_KEY = "9e9ab06256174d82a01dc73f02418ca2"

def obtener_juegos_por_anio(inicio=2015, fin=2024, paginas=5, pausa=1):
    juegos_lista = []

    for year in tqdm(range(inicio, fin + 1), desc="Años"):
        for page in tqdm(range(1, paginas + 1), desc=f"Año {year}", leave=False):
            url = (
                f"https://api.rawg.io/api/games"
                f"?key={API_KEY}&dates={year}-01-01,{year}-12-31"
                f"&page_size=40&page={page}"
            )
            try:
                r = requests.get(url)
                r.raise_for_status()  # Lanza excepción si hay error
                juegos = r.json().get('results', [])
                if not juegos:
                    print(f"Sin resultados en {year}, página {page}")
                    break
                juegos_lista.extend(juegos)
            except requests.RequestException as e:
                print(f"Error en {year}, página {page}: {e}")
                continue

            time.sleep(pausa)  # Pausa para no saturar la API

    df = pd.json_normalize(juegos_lista)
    print(f"\nTotal de juegos obtenidos: {len(df)}")
    return df


In [ ]:


def juegos_json(inicio=2015, fin=2024, paginas=5, pausa=1, path="../data/raw/juegos_rawg.json"):

    print("Obteniendo datos desde la API de RAWG...")
   
    df = obtener_juegos_por_anio(inicio=inicio, fin=fin, paginas=paginas, pausa=pausa)

    # Guardar en CSV
    # df.to_csv(path, index=False)
    # print(f"Archivo guardado en: {path}")

     # Guardar en JSON
    df.to_json(path, index=False)
    print(f"Archivo guardado en: {path}")





In [ ]:
juegos_json(inicio=2015, fin=2024, paginas=5, pausa=1, path="../data/raw/juegos_rawg.json")
